# *Data collection Notebook*

## Contents and purpose

- Import packages
- set up directory and path structure
- Load raw data from Kaggle and save it to repo
- sift through the data and process/ save it respectively
- clean data
- create train, test and validation sets

## Important files

- kaggle JSON file is a personal authentication token, if this repo is forked and reproduced, it needs to be replaced by an individual file.

## Expected Results

- we will receive the necessary data for the subsequent notebooks
    - a train set to train our models
    - a test set
    - a validation set
- each set will have healthy and afflicted sample images

## Why are we doing this

These steps are common practice for the necessary preparation of data sets for machine learning.


# Install/ Import packages necessary for this notebook

- if you have created your working environment based on the requirements.txt file, you can skip the next step, as the requirements will already be satisfied. If not, you cann install the necessary packages now.

In [ ]:
! pip install -r ../requirements.txt

Now you can import the packages that will be needed in this notebook.

In [ ]:
import os
import numpy


## Set working directory and file path architecture for notebook
As the notebooks are set in a subfolder of this repo we need to adjust the working directory so files can be accessed properly. 

First we check our current working directory.

In [ ]:

current_dir = os.getcwd()
current_dir

Now we can change the directory to the parent folder that contains the complete repo. We will also print our new working directory so we can check everything worked out as planned.

In [ ]:
os.chdir('../')
print("You set a new current directory:")
print(os.getcwd())